## 1. 讀入需要的套件

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# For data preprocessing
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

## 2. 下載並整理 Fashion MNIST 資料集

In [2]:
# Load Fashine MNIST
(x_train, y_train0), (x_test, y_test0) = fashion_mnist.load_data()

# Reshape size
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

# Normalize the range of featurs
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

## 3. Functional API 實作

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate, add

以下為學生實作的流程圖

<img style="float: middle;" src="https://i.imgur.com/RE3HeiC.jpg" width="100%">

$${f_1} \colon \mathbb{R}^{784} \to \mathbb{R}^{284}$$
$${f_2} \colon \mathbb{R}^{784} \to \mathbb{R}^{500}$$
$${f_3} \colon \mathbb{R}^{500} \to \mathbb{R}^{112}$$
$${f_4} \colon \mathbb{R}^{500} \to \mathbb{R}^{388}$$
$${f_5} \colon \mathbb{R}^{396} \to \mathbb{R}^{62}$$
$${f_6} \colon \mathbb{R}^{450} \to \mathbb{R}^{10}$$

In [9]:
f_1 = Dense(284, activation = 'sigmoid')
f_2 = Dense(500, activation = 'sigmoid')
f_3 = Dense(112, activation = 'sigmoid')
f_4 = Dense(388, activation = 'sigmoid')
f_5 = Dense(62, activation = 'sigmoid')
f_6 = Dense(10, activation = 'softmax')

In [10]:
x = Input(shape=(784,))

In [11]:
print(x)

Tensor("input_2:0", shape=(?, 784), dtype=float32)


In [12]:
h_1 = f_1(x)
h_2 = f_2(x)

In [13]:
i_1 = f_3(h_2)
i_2 = f_4(h_2)

In [14]:
j = concatenate([h_1, i_1])
k = f_5(j)
l = concatenate([k, i_2])
y = f_6(l)

In [15]:
print(y)

Tensor("dense_11/Softmax:0", shape=(?, 10), dtype=float32)


In [16]:
model_try = Model(x, y)
model_try.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 500)          392500      input_2[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 284)          222940      input_2[0][0]                    
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 112)          56112       dense_7[0][0]                    
______________________________________________________________________________________________

In [18]:
model_try.compile(loss='mse', optimizer=Adam(), metrics=['accuracy'])

In [21]:
model_try.fit(x_train, y_train, batch_size=128, epochs=4)

Epoch 1/4
60000/60000 [==============================] - 6s 93us/sample - loss: 0.0176 - acc: 0.8793
Epoch 2/4
60000/60000 [==============================] - 6s 95us/sample - loss: 0.0168 - acc: 0.8844
Epoch 3/4
60000/60000 [==============================] - 6s 99us/sample - loss: 0.0159 - acc: 0.8909
Epoch 4/4
60000/60000 [==============================] - 6s 101us/sample - loss: 0.0155 - acc: 0.8940


In [22]:
model_try.save_weights('MNIST_Model_API.h5')

In [23]:
score = model_try.evaluate(x_train, y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

60000/60000 [==============================] - 1s 23us/sample - loss: 0.0152 - acc: 0.8968
Loss: 0.015174633202453455
準確率: 89.67666625976562


## 4. 結論


### 在這個資料集中分層訓練好像沒什麼太大的功用，可能再更複雜的資料集上使用效果會比較顯著